In [1]:
import  tensorflow as tf
from    tensorflow import keras
from    tensorflow.keras import layers, Model
from dataset import make_anime_dataset
import glob
import numpy as np

此处利用直接编写好的 `make_anime_dataset` 函数

In [2]:
batch_size = 64
z_dim = 100
epochs = 100
lr = 1e3
is_training = True

In [3]:
# 设置本地数据集路径
img_path = glob.glob(r'E:\GAN_training_datasets\archive\data\*.png')
dataset, img_shape, _ = make_anime_dataset(img_path, batch_size, resize=64)

由 5 个转置卷积层组成，实现特征图高宽的放大，特征图通道层层减少。

- 将长度为 100 的隐藏变量 $z$ 变换为 [b, 1, 1, 100]
- 依次通过转置卷积层，放大高宽维度，减少通道维度，最后高宽为 64×64，通道为 3
- 每个卷积层中插入 BN 层提高训练稳定性，卷积层选择不使用偏置向量

<img src="img/01.jpg"/>

In [ ]:
# 生成器网络类
class Generator(keras.Model):
    # [b, 100] => [b, 64, 64, 3]
    
    def __init__(self):
        super(Generator, self).__init__()
        filter = 64
        # 输出 channel 为 filter*8,核大小为4， 步长为1，不使用padding和偏置
        self.conv1 = layers.Conv2DTranspose(filter*8, 4, 1, 'valid', use_bias=False)
        self.bn1 = layers.BatchNormalization()

        self.conv1 = layers.Conv2DTranspose(filter*4, 4, 2, 'same', use_bias=False)
        self.bn2 = layers.BatchNormalization()
        # 转置卷积层3
        self.conv3 = layers.Conv2DTranspose(filter*2, 4,2, 'same', use_bias=False)
        self.bn3 = layers.BatchNormalization()
        # 转置卷积层4
        self.conv4 = layers.Conv2DTranspose(filter*1, 4,2, 'same', use_bias=False)
        self.bn4 = layers.BatchNormalization()
        # 转置卷积层5
        self.conv5 = layers.Conv2DTranspose(3, 4,2, 'same', use_bias=False)
    # 
    def call(self, inputs, training = None):
        # [z, 100]
        x = inputs
        # [b, 1, 1, 100]
        x = tf.reshape(x, (x.shape[0], 1, 1, x.shape[1]))
        # 激活函数
        x = tf.nn.relu(x)
        
        

In [ ]:
# 分类器，普通意义上的分类网络
class Discriminator(Model):
    def __init__(self):
        super(Discriminator, self).__init__()
        filter = 64
        # [b, 64, 64, 3]=>[b, 1]
        # filter:64, kernek:5, stride:3
        self.conv1 = layers.Conv2D(filter, 4, 2, 'valid', use_bias=False)
        self.bn1 = layers.BatchNormalization()
        # 使得GAN training 更加稳定
        self.conv2 = layers.Conv2D(filter*2, 4, 2, 'valid', use_bias=False)
        self.bn2 = layers.BatchNormalization()

        self.conv3 = layers.Conv2D(filter*4, 4, 2, 'valid', use_bias=False)
        self.bn3 = layers.BatchNormalization()

        self.conv4 = layers.Conv2D(filter*8, 3, 1, 'valid', use_bias=False)
        self.bn4 = layers.BatchNormalization()
        
        self.conv5 = layers.Conv2D(filter*16, 3, 1, 'valid', use_bias=False)
        self.bn5 = layers.BatchNormalization()

        # 全局池化层
        self.pool = layers.GlobalAveragePooling2D()
        # 特征打平
        self._flatten = layers.Flatten()
        # 全连接层分类
        self.fc = layers.Dense(1)

    
    def call(self, inputs, training=None):
        # (4, 31, 31, 64)
        x = tf.nn.leaky_relu(self.bn1(self.conv1(inputs), training=training))
        # (4, 14, 14, 128)
        x = tf.nn.leaky_relu(self.bn2(self.conv2(x), training=training))
        # (4, 6, 6, 256)
        x = tf.nn.leaky_relu(self.bn3(self.conv3(x), training=training))
        # (4, 4, 4, 512)
        x = tf.nn.leaky_relu(self.bn4(self.conv4(x), training=training))
        # 卷积-BN-激活函数:(4, 2, 2, 1024)
        x = tf.nn.leaky_relu(self.bn5(self.conv5(x), training=training))
        # 卷积-BN-激活函数:(4, 1024)
        x = self.pool(x)
        # 打平
        x = self.flatten(x)
        # 输出，[b, 1024] => [b, 1]
        logits = self.fc(x)

        return logits


        

